In [18]:
import os
import stata_setup
## 设置pystata目录
stata_setup.config(os.getenv("STATA_SYSDIR"), 'mp')
from pystata import stata

本章演示用的数据来自任胜钢等学者2019年发表的一项关于中国二氧化硫排放权交易试点政策的评估研究。2007年，中国开始实行二氧化硫排放权交易试点政策，先后批复了江苏、天津、浙江、河北、山西等11个排放权交易试点省份，但还有很多省份没有作为试点地区。任胜钢等学者收集了554家上市企业在实施试点政策前后多年的全要素增长率，并使用双重差分法、三重差分法等估计了排污权对上市企业全要素生产率的影响。

数据so2.dta包含了他们进行这项研究所使用的主要变量。其中，company为企业代码，year为观测年份，lntfp为全要素生产率的对数值（它是研究的因变量），so2是标识该企业是否排放二氧化硫的虚拟变量（so2=1表示排放二氧化硫，so2=0表示不排放二氧化硫），treat是标识是否为试点地区的虚拟变量（treat=1表示是试点地区，treat=0 表示不是试点地区）。该数据中的其他变量是研究可能用到的协变量。

In [29]:
%stata use so2.dta, clear

In [27]:
stata.run('set linesize 180')

In [30]:
%stata list in 1/5


     +------------------------------------------------------------------------------------------------------------------------+
     | company   area   year      lntfp   so2   treat    zcsy        lf   age   owner      sczy       lnaj    lnlabor   lnzlb |
     |------------------------------------------------------------------------------------------------------------------------|
  1. |       8      2   2004   5.933585     0       0    77.4   .545455    16       0   .005587    6.77308   3.295837       0 |
  2. |       8      2   2005    1.75512     0       0    -7.5         0    17       0   .003786    6.50129    5.42495       0 |
  3. |       8      2   2006   3.144548     0       0   -2.49         0    18       0   .000639   6.986567   3.218876       0 |
  4. |       8      2   2007   5.288451     0       0    2.32         1    19       0   .000463   7.527794   3.091043       0 |
  5. |       8      2   2008   4.986861     0       0    1.25         0    20       0   .000443   7.858

In [31]:
%stata tab year


       year |      Freq.     Percent        Cum.
------------+-----------------------------------
       2004 |        554        8.33        8.33
       2005 |        554        8.33       16.67
       2006 |        554        8.33       25.00
       2007 |        554        8.33       33.33
       2008 |        554        8.33       41.67
       2009 |        554        8.33       50.00
       2010 |        554        8.33       58.33
       2011 |        554        8.33       66.67
       2012 |        554        8.33       75.00
       2013 |        554        8.33       83.33
       2014 |        554        8.33       91.67
       2015 |        554        8.33      100.00
------------+-----------------------------------
      Total |      6,648      100.00


In [32]:
## 生成标识政策实施的时期变量
stata.run(
    """
    gen time=1 if year>2007
    replace time=0 if year<=2007
    """
)


. 
.     gen time=1 if year>2007
(2,216 missing values generated)

.     replace time=0 if year<=2007
(2,216 real changes made)

.     
. 


在使用双重差分法之前，我们可以先比较一下政策实施之后，试点地区的企业与非试点地区的企业在全要素生产率上的差异。这可以通过以下命令实现。在该命令中，我们将样本限定为政策实施以后排放二氧化硫的企业；此外，考虑到同一个地区的企业可能存在自相关性，我们使用聚类稳健标准误

In [33]:
## 一重差分
%stata reg lntfp treat if time==1 & so2==1, vce(cluster area)


Linear regression                               Number of obs     =      2,320
                                                F(1, 30)          =       0.58
                                                Prob > F          =     0.4525
                                                R-squared         =     0.0018
                                                Root MSE          =     1.0221

                                  (Std. err. adjusted for 31 clusters in area)
------------------------------------------------------------------------------
             |               Robust
       lntfp | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
       treat |   .0888057   .1166604     0.76   0.452    -.1494467     .327058
       _cons |   6.307181   .1065014    59.22   0.000     6.089676    6.524686
------------------------------------------------------------------------------


分析结果显示，在政策实施后，试点地区的二氧化硫排放企业与非试点地区的企业在全要素增长率上并无显著差异。不过，考虑到政策试点并不是随机分配的，直接比较两类企业存在较大偏差。因此，考虑使用双重差分法

我们先演示使用 regress 命令实现双重差分的方法，具体如下:

In [35]:
%stata reg lntfp treat##time if so2==1, vce(cluster area)


Linear regression                               Number of obs     =      3,480
                                                F(3, 30)          =      34.18
                                                Prob > F          =     0.0000
                                                R-squared         =     0.0274
                                                Root MSE          =     .97167

                                  (Std. err. adjusted for 31 clusters in area)
------------------------------------------------------------------------------
             |               Robust
       lntfp | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     1.treat |  -.1634398   .0997303    -1.64   0.112    -.3671161    .0402366
      1.time |   .2265127   .0537277     4.22   0.000     .1167861    .3362394
             |
  treat#time |
        1 1  |   .2522454   .0749276     3.37   0.002     .0992

根据之前的介绍，输出结果中 treat 和 time 的交互项的回归系数即双重差分估计量。分析结果显示，该系数的估计值为 0.252，在 0.01 的水平下统计显著，因此，通过双重差分法，我们验证了排污权交易试点政策对排污企业的全要素增长率有显著的积极影响。接下来，我们可以在回归方程中纳入协变量，以分析上述研究结论的稳健性：

In [36]:
%stata reg lntfp treat##time zcsy lf age owner sczy lnaj lnlabor lnzlb if so2==1, vce(cluster area)


Linear regression                               Number of obs     =      3,479
                                                F(11, 30)         =      49.21
                                                Prob > F          =     0.0000
                                                R-squared         =     0.2511
                                                Root MSE          =     .85347

                                  (Std. err. adjusted for 31 clusters in area)
------------------------------------------------------------------------------
             |               Robust
       lntfp | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     1.treat |  -.1989419   .1019939    -1.95   0.061    -.4072411    .0093573
      1.time |    .112871   .0446667     2.53   0.017     .0216494    .2040925
             |
  treat#time |
        1 1  |   .2687978   .0693089     3.88   0.001     .1272

上面 regress 命令的输出结果也可以通过 diff 命令获得，具体如下：

In [37]:
## 使用diff
%stata diff lntfp if so2==1, t(treat) p(time) cluster(area)


DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
--------------------------------------------
Number of observations in the DIFF-IN-DIFF: 3480
            Before         After    
   Control: 716            1432        2148
   Treated: 444            888         1332
            1160           2320
--------------------------------------------------------
 Outcome var.   | lntfp   | S. Err. |   |t|   |  P>|t|
----------------+---------+---------+---------+---------
Before          |         |         |         | 
   Control      | 6.081   |         |         | 
   Treated      | 5.917   |         |         | 
   Diff (T-C)   | -0.163  | 0.100   | -1.64   | 0.112
After           |         |         |         | 
   Control      | 6.307   |         |         | 
   Treated      | 6.396   |         |         | 
   Diff (T-C)   | 0.089   | 0.117   | 0.76    | 0.453
                |         |         |         | 
Diff-in-Diff    | 0.252   | 0.075   | 3.37    | 0.002***
--------------------------

In [38]:
## 加入协变量
%stata diff lntfp if so2==1, t(treat) p(time) cluster(area) cov(zcsy lf age owner sczy lnaj lnlabor lnzlb)

DIFFERENCE-IN-DIFFERENCES WITH COVARIATES
-----------------------------------------

DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
--------------------------------------------
Number of observations in the DIFF-IN-DIFF: 3479
            Before         After    
   Control: 715            1432        2147
   Treated: 444            888         1332
            1159           2320
--------------------------------------------------------
 Outcome var.   | lntfp   | S. Err. |   |t|   |  P>|t|
----------------+---------+---------+---------+---------
Before          |         |         |         | 
   Control      | 3.896   |         |         | 
   Treated      | 3.698   |         |         | 
   Diff (T-C)   | -0.199  | 0.102   | -1.95   | 0.061*
After           |         |         |         | 
   Control      | 4.009   |         |         | 
   Treated      | 4.079   |         |         | 
   Diff (T-C)   | 0.070   | 0.094   | 0.74    | 0.464
                |         |         |         |

分析结果显示，在不纳入协变量的情况下，diff 命令得到的双重差分估计值为 0.252；纳入协变量之后，该估计值为 0.269。这与之前通过 regress 命令得到的结果完全相同，因此，这两个命令是完全等价的。不过，使用 diff 命令可以实现更多功能。

In [39]:
## 检测干预前协变量在控制组和对照组之间是否存在显著差异
%stata diff lntfp if so2==1, t(treat) p(time) cluster(area) cov(zcsy lf age owner sczy lnaj lnlabor lnzlb) test

TWO-SAMPLE T TEST
-----------------

Number of observations (baseline): 1160
            Before         After    
   Control: 716            -           716
   Treated: 444            -           444
            1160           -

t-test at period = 0:
----------------------------------------------------------------------------------------------
 Variable(s)         |   Mean Control   | Mean Treated |    Diff.   |   |t|   |  Pr(|T|>|t|)
---------------------+------------------+--------------+------------+---------+---------------
lntfp                | 6.081            | 5.917        | -0.163     |  1.64   | 0.1117
zcsy                 | 7.229            | 7.204        | -0.025     |  0.02   | 0.9858
lf                   | 2.079            | 3.398        | 1.319      |  1.19   | 0.2416
age                  | 11.511           | 11.365       | -0.146     |  0.20   | 0.8407
owner                | 0.615            | 0.586        | -0.029     |  0.43   | 0.6737
sczy                 | 1.236  

分析结果显示，政策实施以前，协变量 sczy 在试点企业和非试点企业之间存在显著差异，但两类企业在其他协变量上的差异并不显著。由此可见，这两类企业总体来说还是比较相似的，这就为论证平行趋势假定奠定了基础

其次，使用 diff 命令还可以通过倾向值匹配法实现对协变量的统计控制。具体来说，我们可以使用与之前相同的协变量，并通过 logit 模型预测倾向值，然后使用核匹配法在共同取值范围内实施匹配

In [40]:
## PSM+双重差分
%stata diff lntfp if so2==1, t(treat) p(time) cluster(area) cov(zcsy lf age owner sczy lnaj lnlabor lnzlb) kernel id(company) logit support

KERNEL PROPENSITY SCORE MATCHING DIFFERENCE-IN-DIFFERENCES
----------------------------------------------------------
    Estimation on common support
    Matching iterations...
....................................................................................................................................................................................
> ..................................................................................................................................................................................
> ..................................................................
DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
--------------------------------------------
Number of observations in the DIFF-IN-DIFF: 3192
            Before         After    
   Control: 656            1288        1944
   Treated: 424            824         1248
            1080           2112
--------------------------------------------------------
 Outcome var.   | lntfp   | S. Err. | 

分析结果显示，进行上述倾向值匹配之后的双重差分估计值为 0.290，这与之前通过线性回归得到的分析结果非常接近。因此，无论使用哪种统计控制方法，研究结论都不变

在实施倾向值匹配双重差分之后，通常要对匹配样本的平衡性进行检验，这可以通过以下命令实现：

In [41]:
## 检测匹配后协变量在干预组和控制组是否有显著差异
%stata diff lntfp if so2==1, t(treat) p(time) cluster(area) cov(zcsy lf age owner sczy lnaj lnlabor lnzlb) kernel id(company) logit support test

    Matching iterations...
....................................................................................................................................................................................
> ..................................................................................................................................................................................
> ..................................................................
TWO-SAMPLE T TEST
-----------------
    Test on common support

Number of observations (baseline): 1160
            Before         After    
   Control: 716            -           716
   Treated: 444            -           444
            1160           -

t-test at period = 0:
----------------------------------------------------------------------------------------------
Weighted Variable(s) |   Mean Control   | Mean Treated |    Diff.   |   |t|   |  Pr(|T|>|t|)
---------------------+------------------+--------------+------------+------

在实施倾向值匹配之后，所有协变量在试点企业和非试点企业之间均不存在显著差异。因此，匹配后的数据平衡性很好。基于倾向值匹配的分析结果是可信的

最后，使用 diff 命令还可以非常方便地获得三重差分估计量。在上述双重差分估计中，我们需要假定在没有政策试点的情况下，试点地区的排污企业与非试点地区的排污企的全要素生产率会同步发生变化，这个假定可能成立，也可能不成立。如果假定不成立，那么一种可能的分析方法是，使用试点地区的非排污企业和非试点地区的非排污企业在全要素生产率变动趋势上的差异对之前双重差分的估计偏差进行校正。这就是三重差分法。具体来说，我们可以将 so2 作为额外的分组变量纳入分析过程，这可以通过 `diff` 命令的 ddd() 选项实现

In [42]:
## 三重差分
%stata diff lntfp, t(treat) p(time) cluster(area) cov(zcsy lf age owner sczy lnaj lnlabor lnzlb) ddd(so2)

TRIPLE DIFFERENCE-IN-DIFFERENCES WITH COVARIATES
------------------------------------------------

TRIPLE DIFFERENCE (DDD) ESTIMATION RESULTS
------------------------------------------
Notation of DDD:
   Control (A)     treat = 0 and so2 = 1
   Control (B)     treat = 0 and so2 = 0
   Treated (A)     treat = 1 and so2 = 1
   Treated (B)     treat = 1 and so2 = 0

Number of observations in the DDD: 6645
               Before      After    
   Control (A):715         1432        2147
   Control (B):659         1319        1978
   Treated (A):444         888         1332
   Treated (B):396         792         1188
               2214        4431
--------------------------------------------------------
 Outcome var.   | lntfp   | S. Err. |   |t|   |  P>|t|
----------------+---------+---------+---------+---------
Before          |         |         |         | 
   Control (A)  | 3.987   |         |         | 
   Control (B)  | 3.807   |         |         | 
   Treated (A)  | 3.809   |     

分析结果显示，三重差分估计值为 0.473，在 0.001 的水平下统计显著。因此，与之前双重差分法的研究结论相同，通过三重差分法也发现，排污权交易试点对排污企业的全要素生产率具有显著的积极影响。

上述三重差分法也可以通过 `regress` 命令实现，具体如下：

In [43]:
%stata reg lntfp treat##time##so2 zcsy lf age owner sczy lnaj lnlabor lnzlb, vce(cluster area)


Linear regression                               Number of obs     =      6,645
                                                F(15, 30)         =     100.09
                                                Prob > F          =     0.0000
                                                R-squared         =     0.2836
                                                Root MSE          =     .85096

                                    (Std. err. adjusted for 31 clusters in area)
--------------------------------------------------------------------------------
               |               Robust
         lntfp | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
---------------+----------------------------------------------------------------
       1.treat |   .0227465   .1024759     0.22   0.826    -.1865372    .2320303
        1.time |   .4652871   .0566712     8.21   0.000     .3495491    .5810252
               |
    treat#time |
          1 1  |  -.2082028    .076687    -2.

在上述输出结果中，treat、time 和 so2 的三维交互项的系数即三重差分估计量，可以发现，其估计值为 0.473，这与上文通过 `diff` 命令得到的结果完全相同。因此，通过这两个命令进行三重差分估计是完全等价的。<br>
接下来，我们将演示如何在多期调查数据中应用双重差分法。如前所述，使用多期调查数据不仅可以更好地分析政策干预的短期效应、长期效应、预期效应和滞后效应，而且可以在一定程度上对平行趋势假定进行检验。这可以通过在回归方程中纳入干预变量与时间变量的交互项实现，具体如下：

In [45]:
## 多期双重差分
%stata reg lntfp treat##i.year if so2==1, vce(cluster area)


Linear regression                               Number of obs     =      3,480
                                                F(21, 30)         =          .
                                                Prob > F          =          .
                                                R-squared         =     0.0493
                                                Root MSE          =     .96342

                                  (Std. err. adjusted for 31 clusters in area)
------------------------------------------------------------------------------
             |               Robust
       lntfp | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     1.treat |  -.1840304   .1127734    -1.63   0.113    -.4143443    .0462835
             |
        year |
       2005  |   .0090056    .046054     0.20   0.846    -.0850492    .1030604
       2006  |   .0954318   .0521879     1.83   0.077    -.0111

分析结果显示，treat 与 year 的交互项的系数在 2005 年、2006 年和 2007 年这三年很小，且在统计上都不显著。因此，可以认为在政策试点之前，试点企业和非试点企业满足平行趋势假定。需要注意的是，在政策试点前满足平行趋势假定不足以证明在政策试点后仍将满足该假定，但毫无疑问的是，上述分析结果为政策试点后两类企业继续满足平行趋势假定奠定了基础。<br>
此外，从分析结果还可以发现，treat 与 year 的交互项的系数从 2008 年起开始上升，但在 2008 年和 2009 年并不显著。这说明，排污权交易试点政策的效应存在一定的滞后性。不过，在 2010 年该政策的效应表现出一定的显著性后就一直稳定存在，因此，可以认为该政策的效应是长期的。

In [46]:
## 加入协变量
%stata reg lntfp treat##i.year zcsy lf age owner sczy lnaj lnlabor lnzlb if so2==1, vce(cluster area)


Linear regression                               Number of obs     =      3,479
                                                F(29, 30)         =          .
                                                Prob > F          =          .
                                                R-squared         =     0.2657
                                                Root MSE          =     .84755

                                  (Std. err. adjusted for 31 clusters in area)
------------------------------------------------------------------------------
             |               Robust
       lntfp | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     1.treat |  -.2282414   .1150343    -1.98   0.056    -.4631729    .0066901
             |
        year |
       2005  |   .0891487   .0572108     1.56   0.130    -.0276912    .2059887
       2006  |   .0992199   .0591842     1.68   0.104    -.0216

练习

使用 Stata 软件打开数据文件 `ada.dta`，进行双重差分及其拓展分析。《美国残疾人法案》（Americans with Disabilities Act, ADA）于 1990 年 7 月通过，并于 1992 年 7 月正式生效。美国政府颁布该法案的初衷是保护残疾人在求职、解聘和薪水等方面不受歧视，但一些学者担忧，该法案会导致企业雇佣残疾人的成本上升，进而对残疾人就业产生负面影响。为了评估 ADA 对残疾人就业的因果影响，阿西莫格鲁和安格里斯特分析了该法案生效前后美国残疾人就业率的变化。<br>
数据 ada.dta 包含了他们进行这项研究使用的主要数据。
变量说明：
year 是调查年份；
disabled 是标识研究对象是不是残疾人的虚拟变量；
wkswork 是研究对象在过去一年的就业周数，是研究的因变量；
age_G、edu_G、race_G、region 是四个控制变量，分别测量研究对象的年龄组、教育分组、种族类型和所属地区。

In [47]:
%stata use ada.dta, clear

In [48]:
## 生成标识ADA法案生效前后的虚拟变量
stata.run(
    """
    gen post=1 if year>1992
    replace post=0 if year<=1992
    """
)


. 
.     gen post=1 if year>1992
(124,214 missing values generated)

.     replace post=0 if year<=1992
(124,214 real changes made)

.     
. 


In [57]:
data = stata.pdataframe_from_data()
data.head()

,year,disabled,wkswork,age_G,edu_G,race_G,region,post,_Iage_G_2,_Iedu_G_2,...,_Irace_G_2,_Irace_G_3,_Iregion_12,_Iregion_21,_Iregion_22,_Iregion_31,_Iregion_32,_Iregion_33,_Iregion_41,_Iregion_42
0,1987,0,52,2,3,1,11,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1987,0,52,1,3,1,11,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1987,0,52,2,3,1,11,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1987,0,52,2,3,1,11,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1987,0,52,2,2,1,11,0.0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [51]:
## 两期双重差分
%stata reg wkswork i.disabled##i.post
print('\n')
## 纳入协变量
%stata reg wkswork i.disabled##i.post i.age_G i.edu_G i.race_G i.region


      Source |       SS           df       MS      Number of obs   =   195,714
-------------+----------------------------------   F(3, 195710)    =   8417.89
       Model |  5968904.55         3  1989634.85   Prob > F        =    0.0000
    Residual |  46257632.4   195,710  236.358042   R-squared       =    0.1143
-------------+----------------------------------   Adj R-squared   =    0.1143
       Total |    52226537   195,713  266.852672   Root MSE        =    15.374

-------------------------------------------------------------------------------
      wkswork | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
--------------+----------------------------------------------------------------
     disabled |
      Disabl  |  -25.14278   .2063658  -121.84   0.000    -25.54725    -24.7383
       1.post |   .2286792     .07394     3.09   0.002     .0837587    .3735998
              |
disabled#post |
    Disabl#1  |  -2.422757   .3399927    -7.13   0.000    -3.089135    -1.75

In [53]:
## 使用diff命令
%stata diff wkswork, treat(disabled) period(post)
print('\n\n')
%stata xi : diff wkswork, treat(disabled) period(post) cov(i.age_G i.edu_G i.race_G i.region)


DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
--------------------------------------------
Number of observations in the DIFF-IN-DIFF: 195714
            Before         After    
   Control: 118391         68101       186492
   Treated: 5823           3399        9222
            124214         71500
--------------------------------------------------------
 Outcome var.   | wkswork | S. Err. |   |t|   |  P>|t|
----------------+---------+---------+---------+---------
Before          |         |         |         | 
   Control      | 44.515  |         |         | 
   Treated      | 19.373  |         |         | 
   Diff (T-C)   | -25.143 | 0.206   | -121.84 | 0.000***
After           |         |         |         | 
   Control      | 44.744  |         |         | 
   Treated      | 17.179  |         |         | 
   Diff (T-C)   | -27.566 | 0.270   | 102.02  | 0.000***
                |         |         |         | 
Diff-in-Diff    | -2.423  | 0.340   | 7.13    | 0.000***
---------------

In [54]:
## 检验协变量在残疾人与非残疾人之间是否存在显著差异
%stata xi : diff wkswork, treat(disabled) period(post) cov(i.age_G i.edu_G i.race_G i.region) test

i.age_G           _Iage_G_1-2         (naturally coded; _Iage_G_1 omitted)
i.edu_G           _Iedu_G_1-3         (naturally coded; _Iedu_G_1 omitted)
i.race_G          _Irace_G_1-3        (naturally coded; _Irace_G_1 omitted)
i.region          _Iregion_11-42      (naturally coded; _Iregion_11 omitted)
TWO-SAMPLE T TEST
-----------------

Number of observations (baseline): 124214
            Before         After    
   Control: 118391         -           118391
   Treated: 5823           -           5823
            124214         -

t-test at period = 0:
----------------------------------------------------------------------------------------------
 Variable(s)         |   Mean Control   | Mean Treated |    Diff.   |   |t|   |  Pr(|T|>|t|)
---------------------+------------------+--------------+------------+---------+---------------
wkswork              | 44.515           | 19.373       | -25.143    | 121.98  | 0.0000***
_Iage_G_2            | 0.469            | 0.572        | 0.103    

In [55]:
## 多期双重差分
%stata reg wkswork disabled##i.year i.age_G i.edu_G i.race_G i.region


      Source |       SS           df       MS      Number of obs   =   195,714
-------------+----------------------------------   F(32, 195681)   =   1174.58
       Model |  8415310.19        32  262978.444   Prob > F        =    0.0000
    Residual |  43811226.8   195,681  223.891061   R-squared       =    0.1611
-------------+----------------------------------   Adj R-squared   =    0.1610
       Total |    52226537   195,713  266.852672   Root MSE        =    14.963

-------------------------------------------------------------------------------
      wkswork | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
--------------+----------------------------------------------------------------
     disabled |
      Disabl  |  -23.16538   .4883545   -47.44   0.000    -24.12254   -22.20822
              |
         year |
        1988  |   .2881142   .1522207     1.89   0.058    -.0102347    .5864632
        1989  |   1.723855   .1490184    11.57   0.000     1.431783    2.015